In [28]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'Malgun Gothic'

In [29]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

df = pd.concat([train_data, test_data], ignore_index=True)
df

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160486,TEST_64192,30000000,36 months,3 years,MORTGAGE,78000000,22.08,27,부채 통합,2,1307532,763380.0,0.0,0.0,NaN
160487,TEST_64193,30000000,60 months,10+ years,MORTGAGE,109200000,12.06,26,부채 통합,0,960612,1245252.0,0.0,0.0,NaN
160488,TEST_64194,6120000,36 months,10+ years,RENT,39600000,28.80,33,부채 통합,0,131520,80880.0,0.0,0.0,NaN
160489,TEST_64195,11520000,36 months,10+ years,MORTGAGE,66000000,25.44,41,부채 통합,1,1339536,601872.0,0.0,0.0,NaN


- 전처리

    - 근로기간 
      - 범주형 묶어주기 10+, <1 기호 살리기 ✅

    - 주택소유상태 
      - "ANY"인 것 삭제 ✅

    - 부채_대비_소득_비율
      - 9999 일단 두고 나중에 제거한 버전이랑 비교

    - 대출기간
      - months 제거하고 수치형으로 ✅

    - 범주형 데이터 전처리
      - 대출목적 → 레이블인코딩 ✅
      - 주택소유상태 → 레이블인코딩 ✅
      - 근로기간 → 레이블인코딩 ✅


- 근로기간 (범주형 묶어주기 10+, <1 기호 살리기)

In [30]:
df['근로기간'] = df['근로기간'].apply(lambda x: x + ' years' if x.isdigit() else x)
df['근로기간'] = df['근로기간'].replace('1 year', '1 years')
df['근로기간'] = df['근로기간'].str.replace(' ', '')
df['근로기간'] = df['근로기간'].str.replace('years|year', '', regex=True) #'years' 또는 'year' 제거

pd.pivot_table(df, index = '근로기간', aggfunc='size')

근로기간
1          10579
10+        54034
2          14059
3          12755
4           9250
5           9592
6           6353
7           6329
8           8172
9           6289
<1         13546
Unknown     9533
dtype: int64

- 주택소유상태 ("ANY"인 것 삭제)

In [31]:
df = df[df['주택소유상태'] != 'ANY']

- 대출기간 (months 제거하고 수치형으로)

In [32]:
# 제거 후
df['대출기간'] = df['대출기간'].str.replace(' months', '')

#pd.pivot_table(df, index = '대출기간', aggfunc='size')

C:\Users\lyr05\AppData\Local\Temp\ipykernel_20732\325134166.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['대출기간'] = df['대출기간'].str.replace(' months', '')


파생변수   
a. 총상환액 = 총상환원금 + 총상환이자  
b. 총상환대출비율 = 총상환액 / 대출금액  
c. 월별대출금액 = 대출금액/대출기간 

In [33]:
df['총상환액'] = df['총상환원금'] + df['총상환이자']

C:\Users\lyr05\AppData\Local\Temp\ipykernel_20732\3318128802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['총상환액'] = df['총상환원금'] + df['총상환이자']


In [34]:
df['총상환대출비율'] = df['총상환액'] / df['대출금액']

C:\Users\lyr05\AppData\Local\Temp\ipykernel_20732\1806991496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['총상환대출비율'] = df['총상환액'] / df['대출금액']


In [35]:
#파생변수 '월별대출금액'을 만드는 데 에러 발생 → 대출기간에 대해 에러가 있는 것으로 보임
print(df['대출기간'].unique()) #→ '대출기간'열 값 확인
df['대출기간'] = pd.to_numeric(df['대출기간'], errors='coerce') #→ 문자열 숫자로 변환

[' 36' ' 60']


C:\Users\lyr05\AppData\Local\Temp\ipykernel_20732\4259056225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['대출기간'] = pd.to_numeric(df['대출기간'], errors='coerce') #→ 문자열 숫자로 변환


In [36]:
df['월별대출금액'] = df['대출금액'] / df['대출기간'] 

C:\Users\lyr05\AppData\Local\Temp\ipykernel_20732\4078007686.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['월별대출금액'] = df['대출금액'] / df['대출기간']


In [37]:
# 중간점검 - 파생변수 생성 확인 (160490 rows × 21 columns)
df.head(3)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급,총상환액,총상환대출비율,월별대출금액
0,TRAIN_00000,12480000,36,6,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C,0.0,0.000000,346666.666667
1,TRAIN_00001,14400000,60,10+,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B,607632.0,0.042197,240000.000000
2,TRAIN_00002,12000000,36,5,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A,1080588.0,0.090049,333333.333333


In [38]:
# 수치형 변수 로그변환

numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns

for column in numeric_columns:
    df[column] = np.log1p(df[column])
    
print(df)

                 ID       대출금액      대출기간     근로기간    주택소유상태       연간소득  \
0       TRAIN_00000  16.339638  3.610918        6      RENT  18.092177   
1       TRAIN_00001  16.482739  4.110874      10+  MORTGAGE  18.689180   
2       TRAIN_00002  16.300417  3.610918        5  MORTGAGE  18.379859   
3       TRAIN_00003  16.482739  3.610918        8  MORTGAGE  18.698312   
4       TRAIN_00004  16.705882  4.110874  Unknown      RENT  18.088503   
...             ...        ...       ...      ...       ...        ...   
160486   TEST_64192  17.216708  3.610918        3  MORTGAGE  18.172219   
160487   TEST_64193  17.216708  4.110874      10+  MORTGAGE  18.508692   
160488   TEST_64194  15.627073  3.610918      10+      RENT  17.494340   
160489   TEST_64195  16.259595  3.610918      10+  MORTGAGE  18.005165   
160490   TEST_64196  15.607270  3.610918        9      RENT  19.008467   

        부채_대비_소득_비율      총계좌수   대출목적  최근_2년간_연체_횟수      총상환원금      총상환이자  \
0          2.990720  2.772589  부채 통

C:\Users\lyr05\AppData\Local\Temp\ipykernel_20732\2432167627.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.log1p(df[column])


- train/test 데이터셋 분할

In [39]:
train_data = df.dropna(subset=['대출등급'])
test_data = df[df['대출등급'].isnull()]
test_data = test_data.drop(columns=['대출등급'])

In [40]:
# 96293 rows × 18 columns
train_data.head(3)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급,총상환액,총상환대출비율,월별대출금액
0,TRAIN_00000,16.339638,3.610918,6,RENT,18.092177,2.990720,2.772589,부채 통합,0.0,0.000000,0.000000,0.0,0.0,C,0.000000,0.000000,12.756122
1,TRAIN_00001,16.482739,4.110874,10+,MORTGAGE,18.689180,3.149740,3.091042,주택 개선,0.0,12.830869,12.363337,0.0,0.0,B,13.317326,0.041331,12.388398
2,TRAIN_00002,16.300417,3.610918,5,MORTGAGE,18.379859,2.261763,2.708050,부채 통합,0.0,13.741482,11.931274,0.0,0.0,A,13.893017,0.086223,12.716901


In [41]:
# 64197 rows × 17 columns
test_data.head(3)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,총상환액,총상환대출비율,월별대출금액
96294,TEST_00000,16.636890,3.610918,8,MORTGAGE,18.698312,3.027231,2.564949,주택 개선,0.0,12.885864,11.895497,0.0,0.0,13.201723,0.031712,13.053373
96295,TEST_00001,15.943742,3.610918,5,RENT,18.315000,2.823757,3.258097,부채 통합,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,12.360228
96296,TEST_00002,16.665060,3.610918,6,RENT,18.826146,2.241773,3.044522,신용 카드,0.0,14.396038,12.549027,0.0,0.0,14.542480,0.113081,13.081543


In [47]:
numeric_columns = train_data.select_dtypes(include=['int64', 'float64']).columns

In [55]:
# Minmax() 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_data = scaler.fit_transform(train_data[numeric_columns])

scaled_mm = pd.DataFrame(scaled_data, columns=numeric_columns)

print(scaled_mm)

           대출금액  대출기간      연간소득  부채_대비_소득_비율      총계좌수  최근_2년간_연체_횟수  \
0      0.658671   0.0  0.783116     0.324713  0.329845      0.000000   
1      0.698921   1.0  0.808957     0.341979  0.420151      0.000000   
2      0.647640   0.0  0.795568     0.245568  0.311543      0.000000   
3      0.698921   0.0  0.809352     0.301639  0.329845      0.000000   
4      0.761684   1.0  0.782957     0.355360  0.393123      0.000000   
...         ...   ...       ...          ...       ...           ...   
96288  0.698921   0.0  0.829450     0.253525  0.543598      0.000000   
96289  0.893880   1.0  0.809352     0.197395  0.467524      0.000000   
96290  0.698921   0.0  0.789788     0.271945  0.432757      0.000000   
96291  0.721434   0.0  0.779565     0.315613  0.420151      0.319923   
96292  0.555243   0.0  0.767677     0.276802  0.311543      0.000000   

          총상환원금     총상환이자  총연체금액  연체계좌수      총상환액   총상환대출비율    월별대출금액  
0      0.000000  0.000000    0.0    0.0  0.000000  0.000000  0.

- 범주형 데이터 인코딩 → 레이블인코딩

In [62]:
from sklearn.preprocessing import LabelEncoder

# 범주형 변수 추출
categorical_cols = ['대출목적', '주택소유상태', '근로기간']
categorical_df = train_data[categorical_cols]

# 라벨 인코딩 적용
label_encoder = LabelEncoder()
categorical_encoded = categorical_df.apply(label_encoder.fit_transform)

In [64]:
scaled_mm.reset_index(drop=True, inplace=True)
categorical_encoded.reset_index(drop=True, inplace=True)

# 데이터셋 합치기
train_data_mm = pd.concat([scaled_mm, categorical_encoded], axis=1)

train_data_mm

,대출금액,대출기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,총상환액,총상환대출비율,월별대출금액,대출목적,주택소유상태,근로기간
0,0.658671,0.0,0.783116,0.324713,0.329845,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.658669,1,2,6
1,0.698921,1.0,0.808957,0.341979,0.420151,0.000000,0.731015,0.795184,0.0,0.0,0.758338,0.057717,0.555240,10,0,1
2,0.647640,0.0,0.795568,0.245568,0.311543,0.000000,0.782895,0.767395,0.0,0.0,0.791120,0.120407,0.647638,1,0,5
3,0.698921,0.0,0.809352,0.301639,0.329845,0.000000,0.723224,0.767885,0.0,0.0,0.744785,0.045690,0.698919,1,0,8
4,0.761684,1.0,0.782957,0.355360,0.393123,0.000000,0.703018,0.766117,0.0,0.0,0.731233,0.028984,0.618003,8,2,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96288,0.698921,0.0,0.829450,0.253525,0.543598,0.000000,0.785646,0.842988,0.0,0.0,0.808519,0.135453,0.698919,3,0,1
96289,0.893880,1.0,0.809352,0.197395,0.467524,0.000000,0.756443,0.878515,0.0,0.0,0.807424,0.068079,0.750200,10,0,1
96290,0.698921,0.0,0.789788,0.271945,0.432757,0.000000,0.809799,0.797126,0.0,0.0,0.817931,0.158464,0.698919,3,0,0
96291,0.721434,0.0,0.779565,0.315613,0.420151,0.319923,0.805396,0.875670,0.0,0.0,0.831513,0.183953,0.721432,1,0,5


In [70]:
train_data_mm.to_csv('train_data_mm.csv', index = False)

In [71]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [72]:
# 대출목적 (le)
test_data['대출목적'] = le.fit_transform(test_data[['대출목적']])

# 주택소유상태 (le)
test_data['주택소유상태'] = le.fit_transform(test_data[['주택소유상태']])

# 근로기간 (le)
test_data['근로기간'] = le.fit_transform(test_data[['근로기간']])

c:\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [73]:
# 중간 점검 - train 범주형 데이터 변환 확인 (64197 rows × 17 columns)
test_data.head(3)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,총상환액,총상환대출비율,월별대출금액
96294,TEST_00000,16.636890,3.610918,8,0,18.698312,3.027231,2.564949,11,0.0,12.885864,11.895497,0.0,0.0,13.201723,0.031712,13.053373
96295,TEST_00001,15.943742,3.610918,5,2,18.315000,2.823757,3.258097,2,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,12.360228
96296,TEST_00002,16.665060,3.610918,6,2,18.826146,2.241773,3.044522,4,0.0,14.396038,12.549027,0.0,0.0,14.542480,0.113081,13.081543


In [74]:
test_data.to_csv('test_data_mm.csv', index = False)